In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import cv2 as cv
import mediapipe as mp

import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from IPython.display import display, Image

from sklearn.preprocessing import MinMaxScaler, Normalizer

from tqdm import tqdm
import os
import sys

import json
from glob import glob
from collections import OrderedDict


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Create Dataloaders

# Define the Neural Network

In [2]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        # First fully connected layer
        self.fc1 = nn.Linear(input_size, hidden_size)  
        # Activation function
        self.relu = nn.ReLU()
        # Output layer
        self.fc2 = nn.Linear(hidden_size, num_classes)  

    def forward(self, x):
        # Forward pass: compute predicted y
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define model parameters
input_size = 63
hidden_size = 128  # You can adjust this value
num_classes = 24

# Instantiate the model
model = NeuralNet(input_size, hidden_size, num_classes)

# Define Loss Function and Optimizer

In [3]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # lr is the learning rate

# Training the Model

In [6]:
def train(model, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

# You would call this function with your data loader
# train(model, your_data_loader, epochs=10)

# Evaluation

In [7]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Call this with your test data loader
# test(model, your_test_data_loader)

# test

In [8]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pandas import DataFrame

In [12]:
df = pd.read_parquet("processed_data.parquet")

column_names = [['label'] + ['path'] + [f'x{x}' for x in range(21)] + [f'y{x}' for x in range(21)] + [f'z{x}' for x in range(21)]]
df.columns = column_names

df.head()

,label,path,x0,x1,x2,x3,x4,x5,x6,x7,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20
0,A,data/images/A/001.jpg,0.588086,0.610797,0.594727,0.589181,0.620655,0.563926,0.560471,0.561863,...,0.701379,0.777772,0.836724,0.704725,0.748712,0.801369,0.809782,0.766050,0.759826,0.758081
1,A,data/images/A/002.jpg,0.623728,0.722062,0.783923,0.794917,0.834841,0.741979,0.736999,0.720012,...,0.632610,0.722303,0.801031,0.628543,0.714150,0.805167,0.785356,0.697666,0.742781,0.785058
2,A,data/images/A/003.jpg,0.459312,0.492102,0.509328,0.532466,0.549604,0.532722,0.506567,0.482298,...,0.661588,0.729322,0.791247,0.704372,0.733751,0.768470,0.785318,0.770730,0.752740,0.741086
3,A,data/images/A/004.jpg,0.435375,0.477092,0.480671,0.489050,0.537486,0.463724,0.454008,0.441640,...,0.767333,0.830541,0.911144,0.767338,0.821366,0.865663,0.886155,0.825251,0.835758,0.835150
4,A,data/images/A/005.jpg,0.564398,0.656270,0.711758,0.731674,0.777790,0.675478,0.685107,0.659716,...,0.637060,0.723884,0.775409,0.623136,0.704702,0.789816,0.748462,0.677460,0.718147,0.756193


In [77]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# # Assuming df is your DataFrame
# le = LabelEncoder()

# # Fit and transform your DataFrame's last column
# df['label'] = le.fit_transform(df.iloc[:, -1])

In [78]:
# df.head()

In [79]:
# from sklearn.model_selection import train_test_split

# # Split the data into features (X) and labels (y)
# X = df.iloc[:, :-1]  # First 63 columns as features
# y = df['label']  # Last column as the label

# # Split the dataset into training set and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader

# # Convert to tensors
# train_tensors = TensorDataset(torch.tensor(X_train.values, dtype=torch.float), torch.tensor(y_train.values, dtype=torch.long))
# test_tensors = TensorDataset(torch.tensor(X_test.values, dtype=torch.float), torch.tensor(y_test.values, dtype=torch.long))

# # Create DataLoaders
# batch_size = 32  # Choose according to your needs or performance considerations
# train_loader = DataLoader(train_tensors, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_tensors, batch_size=batch_size, shuffle=False)

In [18]:


# Assuming your DataFrame is named 'df'
# Convert output strings to integers
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Split features and target
# X = df.iloc[:, :63].values  # Features
# y = df['label'].values  # Target

# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(None, None, test_size=0.2, random_state=42)

# Convert to float32 for better compatibility with PyTorch
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

/home/agustin/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: Expected sequence or array-like, got <class 'NoneType'>

In [16]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y).long()  # Long tensor for classification labels

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [83]:
# Create datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [84]:
train(model, train_loader, 40)
test(model, test_loader)

Train Epoch: 0 [0/6612 (0%)]	Loss: 3.180721
Train Epoch: 0 [3200/6612 (48%)]	Loss: 3.147403
Train Epoch: 0 [6400/6612 (97%)]	Loss: 2.978036
Train Epoch: 1 [0/6612 (0%)]	Loss: 2.932304
Train Epoch: 1 [3200/6612 (48%)]	Loss: 2.796500
Train Epoch: 1 [6400/6612 (97%)]	Loss: 2.452566
Train Epoch: 2 [0/6612 (0%)]	Loss: 2.489521
Train Epoch: 2 [3200/6612 (48%)]	Loss: 2.392346
Train Epoch: 2 [6400/6612 (97%)]	Loss: 2.207166
Train Epoch: 3 [0/6612 (0%)]	Loss: 2.013390
Train Epoch: 3 [3200/6612 (48%)]	Loss: 1.849828
Train Epoch: 3 [6400/6612 (97%)]	Loss: 1.876013
Train Epoch: 4 [0/6612 (0%)]	Loss: 1.891037
Train Epoch: 4 [3200/6612 (48%)]	Loss: 1.749223
Train Epoch: 4 [6400/6612 (97%)]	Loss: 1.522879
Train Epoch: 5 [0/6612 (0%)]	Loss: 1.310601
Train Epoch: 5 [3200/6612 (48%)]	Loss: 1.491680
Train Epoch: 5 [6400/6612 (97%)]	Loss: 1.492205
Train Epoch: 6 [0/6612 (0%)]	Loss: 1.482730
Train Epoch: 6 [3200/6612 (48%)]	Loss: 1.011822
Train Epoch: 6 [6400/6612 (97%)]	Loss: 1.216321
Train Epoch: 7 [0/66

In [95]:
# df.iloc[0].tolist()[:-1]

import numpy as np

np.argmax(model(torch.Tensor(df.iloc[0].tolist()[:-1])).detach().numpy())
# model(torch.Tensor(df.iloc[0].tolist()[:-1]))

0

In [109]:
inp = pd.read_json('test.json').iloc[0].to_list()
np.argmax(model(torch.Tensor(inp)).detach().numpy())


9